<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/Topic_Modeling_Reclame_Aqui_with_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling with Latent Dirichlet Allocation (LDA) - Reclame Aqui**

LDA is an unsupervised learning algorithm which is able to learns about the structure of the data by analyzing the relationships between words and documents. LDA is based on the idea that each document in a collection can be represented as a mixture of a small number of topics, and that each topic is a distribution over the words in the vocabulary. The model estimates the probability of each word in a document given the topics present in that document, and it also estimates the probability of each topic given the words in the document.

Reference: BLEI, D. M. 2012 Probabilistic topic models. Communications of the ACM, 55(4): pp.77_84.


### **Load data from [Github](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git)**

In [6]:
!git clone https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git
%cd /content/Topic-Modeling-Reclame-Aqui/
!git pull origin master

fatal: destination path 'Topic-Modeling-Reclame-Aqui' already exists and is not an empty directory.
/content/Topic-Modeling-Reclame-Aqui
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 6), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (9/9), done.
From https://github.com/punkmic/Topic-Modeling-Reclame-Aqui
 * branch            master     -> FETCH_HEAD
   1a99d77..04e025c  master     -> origin/master
Updating 1a99d77..04e025c
Fast-forward
 Topic_Modeling_with_BERTopic_Reclame_aqui.ipynb |  112 +-
 load_corpus.py                                  |    2 +-
 preprocessing.ipynb                             | 1655 +++++++++++++++++++++++
 3 files changed, 1701 insertions(+), 68 deletions(-)
 create mode 100644 preprocessing.ipynb


In [9]:
# load corpus 
!python '/content/Topic-Modeling-Reclame-Aqui/load_corpus.py' 
corpuss

Select one option: (raw, preprocessed): preprocessed
Data loaded to variable name: corpuss


NameError: ignored